# Exercise 02:  Introduction to distributions and basic sampling in CUQIpy

This notebooks describes basic usage of distributions including visualing their PDF/CDF and generating samples.  It also describes how distributions can be equipped with geometry to represent sampling in nontrivial spaces. Finally conditional distributions are demonstrated along with an application of implementing a hierarchical Gibbs sampler.

## Learning objectives of this notebook:
- Set up random variables following uni- and multivariate distributions in CUQIpy.
- Generate samples from distributions and use CUQIpy tools to inspect visually.
- Explain the use of Geometry in distributions and samples.
- \* Set up conditional distributions in CUQIpy - simple and using lambda functions.
- \* Use conditional distributions to set up a Hierarchical Gibbs sampler.

## Table of contents: 
* [1. Normal distribution (univariate)](#Normal)
* [2. Multivariate distributions](#Multivariate)
* [3. Geometry in distribution and Samples](#Geometry)
* [4. Conditional distributions ★](#Conditional)
* [5. Gibbs sampler ★](#Gibbs)

## References
[1] *Bardsley, Johnathan. 2018. Computational Uncertainty Quantification for Inverse Problems. SIAM, Society for Industrial and Applied Mathematics.*




First we need to import any Python packages needed, here Numpy for array computations and matplotlib for plotting.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

We import CUQIpy. In the previous notebook we imported upfront the specific tools we needed, like `from cuqi.distribution import Gaussian` to get the Gaussian distribution from CUQIpy's distribution module. We now simply import the complete package and then specify the complete name such as `cuqi.distribution.Gaussian` when using it. Both approaches are fine, each with pros and cons.

In [ ]:
import sys
sys.path.append("../../cuqipy")

In [ ]:
import cuqi

A few more settings to make the notebook behave nicely:

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## 1. Normal distribution  (univariate)  <a class="anchor" id="Normal"></a> 

The first thing we can do is define a simple normal distribution of a single variable, e.g.,

$$ X \sim \mathcal{N}(0,1^2) $$

This is done using the following syntax:

In [ ]:
X = cuqi.distribution.Normal(mean=0, std=1)

More information on the distribtion can be found by typing `help(X)`. Try that in the next cell:

In [ ]:
# Type code here:
X.logpdf(grid)

Distributions in CUQIpy have commonly used methods that one might expect like *pdf*, *logpdf*, *cdf*, etc. We demonstrate this here by evaluating and plotting the cumulative distribution function (CDF) on an interval:

In [ ]:
X.logpdf??

In [ ]:
X.mean

In [ ]:
np.sum(-np.log(1*np.sqrt(2*np.pi))-0.5*((grid-0)/1)**2)

In [ ]:
grid = np.linspace(-10, 10, 1001)
plt.plot(grid, X.cdf(grid))

CUQIpy distributions also have `sample` method which returns one or more samples from the distribution:

In [ ]:
X.sample()

By default a single sample is returned. More samples can easily be requested:

In [ ]:
s = X.sample(10000)
type(s)

When more than one sample is generated, a CUQIpy `Samples` object is returned. This is essentially an array in which each column contains one sample, and further equipped with a number of methods for example for plotting.

For example one can make a "chain plot", i.e., the sampled values of selected parameter(s) of interest. Here we have a single parameter and with Python being zero-indexed we specify this parameter as follows:

In [ ]:
s.plot_chain(0)

Another possibility is a histogram of the parameter chain: (The keyword arguments are passed directly to the underlying matplotlib `hist` function for full control)

In [ ]:
s.hist_chain(0, bins=100, density=True)

CUQIpy has integrated support for statistical plotting with the ArviZ library, for a "trace plot" combines the previous two plots:

In [ ]:
s.plot_trace()

and a "violin plot" displays the median as a white circle, the interquartile range, along with the density/histogram on either side:

In [ ]:
s.plot_violin()

#### Try yourself (optional):  
 - Create a new random variable `Y` following a normal distribution with mean 2 and standard deviation 3.
 - Generate 100 samples and display a histogram.
 - Compare with the theoretical distribution by plotting the probability density function of `Y` on top of the histogram.
 - Increase the number of samples and (hopefully) see the histogram approach the theoretical PDF.

In [ ]:
# Type code here:



## 2. Multivariate distributions <a class="anchor" id="Multivariate"></a> 

CUQIpy currently implements a number multivariate distributions in the `cuqi.distribution` module:

- Cauchy_diff
- Gamma
- Gaussian
- GaussianCov
- GaussianPrec
- GaussianSqrtPrec
- GMRF
- Laplace
- Laplace_diff
- LMRF
- Uniform

and more can easily be added when needed.


We here demonstrate using a Gaussian distribution and start by looking at the help:

In [ ]:
help(cuqi.distribution.Gaussian)

We specify here a 3-element random varible `Z` following a Gaussian distribution with independent elements:

$$Z \sim \mathcal{N}(\mu,\mathrm{diag}(\mu^2)) \quad \text{for} \quad \mu = [5, 3, 1]^T$$

In [ ]:
true_mu = np.array([5, 3, 1])
Z = cuqi.distribution.Gaussian(mean=true_mu, std=true_mu)

We generate a single sample which produces a 3-element CUQIarray:

In [ ]:
Z.sample()

If we ask for more than one sample, say 1000, we get a `Samples` object with 1000 columns each holding a 10-element sample:

In [ ]:
sZ = Z.sample(1000)

In [ ]:
sZ.shape

We can plot chains of a few of these variable samples, here we pick element 0 and 2:

In [ ]:
sZ.plot_chain([0,2])

As well as plot a few individal 10-element samples:

In [ ]:
sZ.plot();

By default 5 random samples are plotted, but we can also specify indices of specific samples we wish to plot, like every 100th sample:

In [ ]:
sZ.plot([0, 100, 200, 300, 400, 500, 600, 700, 800, 900]);

We can also plot the sample mean:

In [ ]:
sZ.plot_mean()

and sample standard deviation:

In [ ]:
sZ.plot_std()

#### Try yourself (optional):  
 - Plot mean with 95% credibility interval, hint: `help(plot_ci)`.
 - Include in the credibility interval plot a comparison with the true mean using the `exact` keyword argument of `plot_ci`.
 - Reduce and increase the number of samples and study the effect on the mean and credibility interval.
 - Try also 50% and 99% credibility intervals.

In [ ]:
# Type code here:



## 3. Geometry in distribution and Samples <a class="anchor" id="Geometry"></a> 

By default no particular structure or space is assumed of the parameters. If we want to express that parameters constitute for example a 2D image or are a set of discrete named parameters we can specify this by means of a CUQIpy geometry. 

By default distributions contain a default (trivial) geometry.

In [ ]:
Z.geometry

We may equip the distribution with a different geometry, either when creating it, or afterwards. For example if the five parameters represent labelled quantities such as height, width, depth, weight and density we can use a `Discrete` geometry:

In [ ]:
geom = cuqi.geometry.Discrete(['height','width','depth'])

We can update the distribution's geometry and generate some new samples:

In [ ]:
Z.geometry = geom

In [ ]:
sZ2 = Z.sample(100)

The samples will now know about their new `Discrete` geometry and the plotting style will be changed:

In [ ]:
sZ2.plot();

The credibility interval plot style is also updated to show errorbars for the `Discrete` geometry:

In [ ]:
sZ2.plot_ci(95, exact=true_mu)

And the similarly in the chain plot the legend reflects the particular labels:

In [ ]:
sZ2.plot_chain([0,2])

Another use of geometry is to represent 1D or 2D versions of the same distribution (prior). A Gaussian Markov Random Field (GMRF) can be used in 1 or 2 spatial dimensions, which is represented using `Continuous1D` and `Continuous2D` geometries:

In [ ]:
N = 100     # number of pixels
dom = 2     # 1D or 2D domain

x = np.linspace(0,1,N)

if (dom == 1):
    geometry = cuqi.geometry.Continuous1D(x)
elif (dom == 2):
    geometry = cuqi.geometry.Continuous2D((x, x))

In this example in 1D there will be N parameters and in 2D there will be N^2 parameters. We can check the number of parameters of the geometry as well as its type:

In [ ]:
geometry.dim

In [ ]:
type(geometry)

We can now specify a GMRF distribution (with some chosen mean, precision, boundary conditions etc.) The same exact code will work in 1D and 2D due to the geometry:

In [ ]:
cuqi.distribution.GaussianPrec?

In [ ]:
mean = np.zeros(geometry.dim)
prec = 4
pX = cuqi.distribution.GMRF(mean, prec, N, physical_dim=dom, bc_type='zero', geometry=geometry)
pX = cuqi.distribution.GaussianCov(np.zeros(geometry.dim), cov=2, geometry=geometry)

With the distribution set up, we are ready to generate some samples

In [ ]:
# call method to sample
sampleX = pX.sample(50)

In [ ]:
sampleX.shape

We plot a couple of samples:

In [ ]:
sampleX.plot()   

#### Try yourself (optional):  
 - Go back and change `dom` to 2 to get the 2D case and rerun the subsequent cells.
 - Play with the number of pixels `N` as well as parameters of the GMRF and see the effect on the samples.

## 4. Conditional distributions ★ <a class="anchor" id="Conditional"></a> 

In cuqipy defining conditional distributions is simple. Assume we are interested in defining the Normal distribution condtioned on the standard deviation, e.g.

$$ X_2 \mid \mathrm{std} \sim \mathcal{N}(0,\mathrm{std}^2) $$

This can simply be achieved by *omitting* the keyword argument for the standard deviation as shown in the following code

In [ ]:
X2 = cuqi.distribution.Normal(mean=0)

Because $X_2$ is a conditional distribution, we cannot evaluate the logpdf or sample it directly without specifying the value of the conditioning variable (the standard deviation in this case). Hence the first line in the code cell below would fail.

However, we can specify the conditioning variable using the "call" syntax, i.e., `X2(std=2)` to specify the value of the standard deviation in the conditional distribution as shown below.

In [ ]:
# X2.sample() #This code would fail
X2(std=2).sample()

In general one may need more flexibility than simply conditioning directly on the attributes of the distribution. Let us assume we want to condition on the variance - denoted d - rather than the standard deviation of the normal distribution, i.e.

$$ X_3 \mid d \sim \mathcal{N}(0,d) $$

In cuqipy this is handled by *lambda* functions as follows.

In [ ]:
X3 = cuqi.distribution.Normal(mean=0,std=lambda d: np.sqrt(d))
X3(d=2).sample()

What actually happens behind the scenes is that writing `X3(d=2)` defined a new cuqi distribution, where the standard deviation is defined by evaluating the lambda function. This can be seen by storing the new distribution as follows.

In [ ]:
X4 = X3(d=2)
X4.std

One can even go crazy and define lambda functions for all attributes e.g.

In [ ]:
#Functions for mean and std with various (shared) inputs
mean = lambda sigma,gamma: sigma+gamma
std  = lambda delta,gamma: np.sqrt(delta+gamma)

z = cuqi.distribution.Normal(mean,std)
Z = z(sigma=3,delta=5,gamma=-2)

Z.sample()

### To include
- Use Gaussian before GMRF, with option to choose GMRF instead, maybe others
- Add a bit of theory on GMRFs?
- Update to Image2D instead of Continuous2D?
- Conditional distributions

### Decided not to include in this notebook
- JointDistribution
- Sample from joint distributions
- Is it a joint random variable or joint distribution